# Setup

In [2]:
import json
import os

import openai

try:
    with open(os.path.expanduser("~/.cache/oai"), "r") as f:
        openai.api_key = f.read().strip()
except:
    print("Error reading openai api key from ~/.cache/oai")
    exit(1)

# The Functions API

## The intended use case

As presented by OpenAI, functions in the chat models are meant to be used to get
the model to generate function arguments.

The idea is you ask the model a question and provide it function descriptions,
then if the question seems suitable for a function, the model will generate
the arguments for the function rather than answer the question directly.

The dev can then easily pass the generated arguments to the function and call it,
either to provide the output to the user, or to pass back to the model and get 
another response with the result in context.

\
**TLDR**
- meant to be used to get the model to generate function arguments
- ask the model a question and provide it function descriptions
- if the question seems suitable for a function, the model will generate args
- then pass args to the function
- give result to user, or pass back to model and get another response

To illustrate, here a simplified version of the example OpenAI gives in their docs:

In [3]:
# dummy function to demonstrate
def get_weather(location, unit="c"):
    """Get the weather in a location"""
    # in reality you'd call some api here with the args
    weather = {
        "location": location,
        "unit": unit,
        "temperature": 20,
        "condition": "sunny",
    }
    return weather

In [4]:
msg = "What's the weather like in Kingston?"
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613", # until old models are deprec, use -0613
    messages=[{"role": "user", "content": msg}],
    functions=[  # list of dicts
        {
            "name": "get_weather",  # name of function
            "description": "Get the weather in a location",  # description of function
            "parameters": {  # parameters of function
                "type": "object",  # type of parameters (almost always object)
                "properties": {  # the function arguments
                    "location": {  # arg
                        "type": "string",  # arg type (any json type)
                        "description": "The location to get the weather for",  # arg description
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit to return the temperature in",
                        # can enumerate the possible values instead of leaving openended
                        "enum": ["f", "c"],
                    },
                },
                "required": ["location"],  # list required args
            },
        }
    ],
    function_call="auto",  # "auto" or a function name
)

res_content = res.choices[0].message.to_dict()  # type: ignore
res_content

{'role': 'assistant',
 'content': None,
 'function_call': <OpenAIObject at 0x7f60e7f50cb0> JSON: {
   "name": "get_weather",
   "arguments": "{\n  \"location\": \"Kingston\"\n}"
 }}

when it calls a function, there is nothing in the content field, only function_call

(and vice versa)

so, if we want to then call the function and show the user:

In [5]:
func_to_call = res_content["function_call"]["name"]
args = json.loads(res_content["function_call"]["arguments"])

weather = globals()[func_to_call](**args)
weather

{'location': 'Kingston', 'unit': 'c', 'temperature': 20, 'condition': 'sunny'}

In [6]:
# and so you could show it back to the user like this:
print(
    f"The weather in {args['location']} is {weather['temperature']}°{weather['unit']} and {weather['condition']}"
)

The weather in Kingston is 20°c and sunny


This alone is pretty cool and very useful. Like a more powerful/extensible version of
plugins in chatGPT that we can write ourselves for any usecase. 

Some stuff you guys might want to use it for is using wolfram alpha, wikipedia,
investopedia, or looking up cases or medical information in a database etc.

Sort of just substitutes langchain in a lot of places.

## The more powerful use case

The new versions of the models which have the functions api were of course finetuned 
on tons of data to give these structured argument outputs. So, we can exploit that
to get the model to generate any structured data we want. I don't think I can overstate
how useful this is when trying to get it to conform to some output form, for pretty much
any application.

_The caveat is we need to "trick" it into thinking it's generating function arguments._

Here's how I used it in my work

In [25]:
sys_prompt = """
Before answering, you should think through the question step-by-step.
Explain your reasoning at each step towards answering the question.
If calculation is required, do each step of the calculation as a step in your reasoning.
Finally, indicate the correct answer
"""

question = "let a = b+c. if a = c^2 + 4 and c = b^3 - 7 what is b?"

res = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": question},
    ],
    functions=[
        {
            "name": "answer_question",
            # tell it what the "function" should do
            "description": "Thinks through and answers a multiple choice question on finance",
            "parameters": {
                "type": "object",
                "properties": {  # use args to tell it what fields it should generate
                    "thinking": {
                        "type": "array",  # arrays will be arbitrary length
                        "items": {
                            "type": "string",
                            "description": "Thought and/or calculation for a step in the process of answering the question",
                        },
                        "description": "Step by step thought process and calculations towards answering the question",
                    },
                    "answer": {
                        "type": "string",
                        "description": "The answer to the question",
                        # use enum to restrain its output for easy parsing
#                         "enum": ["A", "B", "C"],
                    },
                },
                "required": ["thinking", "answer"],
            },
        }
    ],
    function_call={"name": "answer_question"},
)
ans = res.choices[0].message.to_dict()["function_call"]["arguments"]  # type: ignore
out = json.loads(ans)
out

{'thinking': ['First, I know that to solve this problem, I have to express all the given equations in terms of b and solve the equation for b.',
  'We have three equations: a = b+c, a = c^2 + 4, and c = b^3 - 7.',
  'From the first equation, we can express a in terms of b: a = b + (b^3 - 7), which simplifies to a = b^4 - 7b + 1.',
  'Substituting this into the second equation, we get: b^4 - 7b + 1 = (b^3 - 7)^2 + 4',
  'After simplifying, we get: b^4 - 7b + 1 = b^6 - 14b^3 + 49 + 4',
  'Subtracting the left hand side from the right hand side gives: 0 = b^6 - b^4 - 14b^3 + 56b - 48',
  'This a complex equation to solve for b and might require numerical methods or factorization to find roots.',
  'Given that this is beyond the scope of the original problem, it is likely that the problem contains a mistake or is incorrectly phrased.'],
 'answer': 'The problem probably contains a mistake or is incorrectly phrased.'}

In [26]:
for i, line in enumerate(out["thinking"]):
    print(f"{i+1}. {line}")

print("\nAnswer:", out["answer"])

1. First, I know that to solve this problem, I have to express all the given equations in terms of b and solve the equation for b.
2. We have three equations: a = b+c, a = c^2 + 4, and c = b^3 - 7.
3. From the first equation, we can express a in terms of b: a = b + (b^3 - 7), which simplifies to a = b^4 - 7b + 1.
4. Substituting this into the second equation, we get: b^4 - 7b + 1 = (b^3 - 7)^2 + 4
5. After simplifying, we get: b^4 - 7b + 1 = b^6 - 14b^3 + 49 + 4
6. Subtracting the left hand side from the right hand side gives: 0 = b^6 - b^4 - 14b^3 + 56b - 48
7. This a complex equation to solve for b and might require numerical methods or factorization to find roots.
8. Given that this is beyond the scope of the original problem, it is likely that the problem contains a mistake or is incorrectly phrased.

Answer: The problem probably contains a mistake or is incorrectly phrased.


In [9]:
answer = "B"
if out["answer"] == answer:
    print("Correct")
else:
    print("Failed")

Correct


### Without functions

In [10]:
# sys_prompt last line: "Finally output the correct answer"
sys_prompt_nofunc = (
    sys_prompt + " in brackets as such: [[answer]] where answer is A, B, or C"
)

res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": sys_prompt_nofunc},
        {"role": "user", "content": question},
    ],
)
print(res.choices[0].message.content)

Before identifying the possible violation, let's first understand what Standard I (B): Independence and Objectivity states. It says that CFA members and candidates must maintain independence and objectivity in their professional activities. They must not offer any services that create a conflict of interest, compromise their independence, or impair their objectivity.

With this in mind, let's review the actions of the manager.

The manager does not want Phil Jones to state any adverse opinions about Alpha One Inc, which is a potential investment banking client.

This action by the manager violates Standard I (B): Independence and Objectivity. It compromises Phil Jones' independence and objectivity in his professional activities. Phil Jones should provide a fair and honest assessment of his research on Alpha One Inc, regardless of the possible impact on their firm’s relations with the company.

Therefore, the correct answer is: [[A]] The manager instructs Phil Jones to issue a favorable

While _most_ of the time it will listen to your formatting request, it's not guaranteed
and often adds some verbose explanation around your formatted output. 

This makes parsing annoyingly inconsistent, and for applications more complicated than this
it may introduce countless edge cases and issues. Overall, its just not a headache 
you need to deal with anymore.

Even for user queries it can be nicer to use functions to get clean structured output
instead of messy inconsistent text. How about getting code help?

In [14]:
question = "In python, calculate the square root of 144"

In [15]:
sys_prompt = """You are a helpful programming assistant that can answer questions about code.
When the question is about syntax or how to do something in a specific language, you should
respond only with the code. Otherwise give an answer as normal.
"""
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": question},
    ],
)
print(res.choices[0].message.content)

Here is the code to calculate the square root of 144 in Python:

```python
import math

result = math.sqrt(144)

print(result)
```

Output:

```python
12.0
```


In [16]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        # {"role": "system", "content": sys_prompt}, # don't even need a sys prompt
        {"role": "user", "content": question},
    ],
    functions=[
        {
            "name": "code_help",
            "description": "Gives the corresponding code for an answer to a question about programming",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "Lines of code constituting the answer to the coding question",
                    },
                },
                "required": ["code"],
            },
        }
    ],
    function_call={"name": "code_help"},
)
ans = res.choices[0].message.to_dict()["function_call"]["arguments"]  # type: ignore
out = json.loads(ans)
print(out["code"])

import math

x = 144

sqrt = math.sqrt(x)

print(sqrt)
